# Experiment

In [ ]:
import numpy as np
import scipy.ndimage
import scipy.signal
import matplotlib.pyplot as plt
from skimage import color, io
from numba import njit
from numba.experimental import jitclass
from numba import int32, float32    # import the types

In [ ]:
spec = [
    ('phi', float32[:,:]), 
    ('dt', float32), # physical time step 
    ('nt', int32), # number of physical time steps
    ('nx', int32), # number of physical time steps
    ('ny', int32), # number of physical time steps
    ('dx', float32), # physical time step
    ('dy', float32), # physical time step
    ('sudot', float32), # time step for pseudo-time iterations
    ('gamma', float32),
    ('max_pseudo_iter', int32),  # maximum pseudo-time iterations
    ('pseudo_tol', float32)  # convergence tolerance for pseudo-time iterations
]

In [ ]:
img = io.imread('data/DRIVE/training/images/21_training.tif')
# img = img[100:-100, 100:-100]
io.imshow(img)
io.show()

In [ ]:
img = img - np.mean(img)
img = color.rgb2gray(img)
img_smooth = scipy.ndimage.filters.gaussian_filter(img, sigma=1)
io.imshow(img_smooth)
io.show()

In [ ]:
def grad(x):
    return np.array(np.gradient(x))
def norm(x, axis=0):
    return np.sqrt(np.sum(np.square(x), axis=axis))
def stopping_fun(x):
    return 1. / (1. + norm(grad(x))**2)

F_v = stopping_fun(img_smooth)
print(img.shape)

In [ ]:
class TimeScheme(object):
	def __init__(self, dt):
		self.dt = dt

	def EulerForward(self, u, rhs, i, j):
		return u[i, j] - self.dt * rhs(u, i, j)



In [ ]:
@jitclass(spec)
class DualTimeStepping:
	def __init__(self, phi, dt, nt, nx, ny, dx, dy, sudot, gamma, max_pseudo_iter, pseudo_tol):
		self.phi = phi
		self.nx = nx
		self.ny = ny
		self.dx = dx
		self.dy = dy
		self.dt = dt  # physical time step
		self.nt = nt  # number of physical time steps
		self.sudot = sudot  # time step for pseudo-time iterations
		self.gamma = gamma
		self.max_pseudo_iter = max_pseudo_iter  # maximum pseudo-time iterations
		self.pseudo_tol = pseudo_tol # convergence tolerance for pseudo-time iterations
		self.initialize()


	def spatialsch(self, u, i, j, ii, jj):
		return u[i, j] - u[ii, jj]

	def initialize(self):
		for i in range(self.nx):
			for j in range(self.ny):
				self.phi[i,j]=np.sqrt((i-self.nx/2)*(i-self.nx/2)+(j-self.ny/2)*(j-self.ny/2))-(self.nx/3)
				# self.phi[i, j] = max(self.nx/4 - i,  i - self.nx/1.5, self.ny/4 - j, j - self.ny/1.5)

	def F(self, phi, i, j):
		return F_v[i,j]

	def rhs(self, phi, i, j):
		delt_xn = self.spatialsch(phi, i, j, i-1, j)
		delt_yn = self.spatialsch(phi, i, j, i, j-1)
		delt_xp = self.spatialsch(phi, i+1, j, i, j)
		delt_yp = self.spatialsch(phi, i, j+1, i, j)
		deltp = max(delt_xn/self.dx, 0)**2 + max(delt_yn/self.dy, 0)**2 + min(delt_xp/self.dx, 0)**2 + min(delt_yp/self.dy, 0)**2
		deltn = max(delt_xp/self.dx, 0)**2 + max(delt_yp/self.dy, 0)**2 + min(delt_xn/self.dx, 0)**2 + min(delt_yn/self.dy, 0)**2
		F = self.F(phi, i,j)
		return max(F, 0) * np.sqrt(deltp) + min(F, 0) * np.sqrt(deltn)

	def residual(self, phi_n, phi_m, phi_nm1, i, j):
		return ((1 + self.gamma) * (phi_m[i, j] - phi_n[i, j]) - self.gamma * (phi_n[i,j] - phi_nm1[i,j]))/self.dt + self.rhs(phi_m, i, j)
	

	def dRdphi(self, phi, i, j):
		# Simplified and corrected Jacobian approximation
		# This is a numerical approximation of the derivative of the residual with respect to phi
		epsilon = 1e-6
		phi_perturbed = phi.copy()
		phi_perturbed[i, j] += epsilon
		
		res1 = self.rhs(phi, i, j)
		res2 = self.rhs(phi_perturbed, i, j)
		
		return (res2 - res1) / epsilon	

	def Coe(self, phi_m, i, j):
		return 1/self.sudot + (1+self.gamma) / self.dt + self.dRdphi(phi_m, i, j) 

	def wrap_func(self, phi_n, phi_m, phi_nm1, i, j):
		return -self.residual(phi_n, phi_m, phi_nm1, i, j) / self.Coe(phi_m, i, j)

	def solve(self):
		phi_m = self.phi.copy()
		phi_n = self.phi.copy()
		phi_nm1 = self.phi.copy()

		for t in range(self.nt):
			# Store previous solutions properly
			phi_nm1 = phi_n.copy()
			phi_n = self.phi.copy()
			
			self.phi = self.apply_boundary_conditions(self.phi)
			phi_m = self.phi.copy()
			
			# Pseudo-time iterations
			for pst in range(self.max_pseudo_iter):
				delta_m = np.zeros((self.nx, self.ny))
				for i in range(1, self.nx-1):
					for j in range(1, self.ny-1):
						delta_m[i, j] = -self.residual(phi_n, phi_m, phi_nm1, i, j) / self.Coe(phi_m, i, j)
				
				# Apply update with relaxation for stability
				phi_m += delta_m
				phi_m = self.apply_boundary_conditions(phi_m)
				
				# Check for convergence
				residual_norm = np.linalg.norm(delta_m)
				if pst % 10 == 0:
					print(f"Pseudo-time step {pst+1}: Residual norm = {residual_norm}")
				if residual_norm < self.pseudo_tol:
					break
			
			self.phi = phi_m.copy()
		return self.phi


			
	
	def apply_boundary_conditions(self, phi):
		# Example boundary conditions (adjust as needed)
		# Zero gradient at boundaries
		phi[0, :] = phi[1, :]
		phi[-1, :] = phi[-2, :]
		phi[:, 0] = phi[:, 1]
		phi[:, -1] = phi[:, -2]
		return phi
	
		

In [ ]:
phi = np.zeros(img.shape, dtype=np.float32)
nx, ny = phi.shape
dx = np.float32(1.0/(nx-1))
dy = np.float32(1.0/(ny-1))
dt = 1
sudo_t = 1e-3
gamma = 0.5
max_pseudo_iter = 2000
pseudo_tol = 1e-4

solver = DualTimeStepping(phi, 1, 20000, nx, ny, dx, dy, sudo_t, gamma, max_pseudo_iter, pseudo_tol)
phi = solver.solve()